In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [46]:

# Load all uploaded files
raw_products_df = pd.read_csv('products.csv')
raw_shedules_df = pd.read_csv('shedules.csv')

# The training data will look like this
# sku_id,	name,	avg_cart,	t_expiery, stocktaking_time,	dt,	trigger


In [66]:
shedules_df = raw_shedules_df[raw_shedules_df['name_TT'].str.contains("5018ДС_СПб_Полюстровский59", case=False, na=False)]

shedules_df = shedules_df.rename(columns={
    'N': 'id',
    'name_TT': 'store',
    'НаименованиеГрафика': 'schedule_name',
    'Длительность (недель)': 'duration_weeks',
    'Стеллажи': 'shelves',
    'неделя 1': 'week_1',
    'неделя 2': 'week_2',
    'неделя 3': 'week_3',
    'неделя 4': 'week_4',
    'Кол-во дней': 'days_count',
    'ЧастотаПостеллажная': 'shelf_frequency',
    'ЧастотаГрафика': 'schedule_frequency'
}, inplace=False)

shedules_df = shedules_df.reset_index(drop=True, inplace=False)

products_df = raw_products_df
products_df['place'] = products_df['place'].apply(lambda x: ''.join(x.split('-')[:2]))

shedules_df.head()['shelves'][0].split(';')

['A1',
 'A10',
 'A10',
 'A10',
 'A10',
 'A10',
 'A10',
 'A10',
 'A10',
 'A11',
 'A11',
 'A11',
 'A11',
 'A12',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'Fr1',
 'Fr1',
 'Fr1',
 'Fr2',
 'Fr3',
 'Fr4',
 'Fr5',
 'Fr6',
 'Fr7',
 'Fr8',
 'Fr9']